# Import libraries

In [1]:
import pandas as pd
import numpy as np
!conda install lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-4.5.0                 |   py36hefd8a0e_0         1.4 MB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    ------------------------------------------------------------
                                           Total:         4.6 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               pkgs/main/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages will be SUPERSEDED by a higher-priority channel:



# Problem 1: Web Scraping

In [85]:
#retrieve any tables from HTML website that have <table> tags
tables = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)

df = tables[0]

# remove not assigned boroughs from the data frame
df = df[df.Borough != "Not assigned"]

# data has no duplicates and neighborhoods are already joined so I just have to reset the index
df = df.reset_index()
df = df.drop('index', axis = 1)
df.shape

(103, 3)

# Problem 2: Preparing Data
## Download csv

In [86]:
!wget http://cocl.us/Geospatial_data

--2020-05-18 11:41:20--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.83, 158.85.108.86, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-18 11:41:20--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-18 11:41:22--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-18 11:41:22--  https://ibm.box.com/public

## Merge both data frames

In [102]:
geo = pd.read_csv("./Geospatial_data")
geo


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [93]:
df_merged = pd.merge(df,geo,how='left')

In [95]:
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Problem 3: Exploring shenanigans
## Import more stuff

In [97]:
from sklearn.cluster import KMeans
import folium
!pip install geopy
from geopy.geocoders import Nominatim

     |████████████████████████████████| 122kB 5.1MB/s eta 0:00:01


## Get langitude and longitude of Toronto

In [98]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Merge data frames

In [99]:
df_ON = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_ON.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


## Place a blue marker on Boroughs with Toronto in their name with Pop up showing their neighborhoods

In [116]:
T_map = folium.Map(location = [latitude,longitude],zoom_start = 12)

for lat, lng, borough, neighborhood in zip(df_ON['Latitude'], df_ON['Longitude'], df_ON['Borough'], df_ON['Neighborhood']):
    popup_label = folium.Popup(str(neighborhood) + ' | ' + str(borough).upper(), parse_html=True)
    folium.CircleMarker(
      [lat,lng],
      radius = 5,
      popup = popup_label, 
      color='blue', 
      fill = True, 
      fill_color = 'blue', 
      fill_opacity = 0.5).add_to(T_map)
T_map